# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116087e+02     1.508152e+00
 * time: 0.3504750728607178
     1     1.028278e+01     9.225900e-01
 * time: 2.1853160858154297
     2    -1.174220e+01     1.074814e+00
 * time: 2.8192861080169678
     3    -3.454695e+01     7.779598e-01
 * time: 3.7571470737457275
     4    -4.811129e+01     5.786060e-01
 * time: 4.693840980529785
     5    -5.727865e+01     2.103109e-01
 * time: 5.628754138946533
     6    -6.003898e+01     1.074885e-01
 * time: 6.302014112472534
     7    -6.101716e+01     5.024535e-02
 * time: 6.932619094848633
     8    -6.138297e+01     7.711410e-02
 * time: 7.554067134857178
     9    -6.167357e+01     3.099491e-02
 * time: 8.18657398223877
    10    -6.185817e+01     2.440849e-02
 * time: 8.83611512184143
    11    -6.199986e+01     2.048309e-02
 * time: 9.469542980194092
    12    -6.205737e+01     2.008302e-02
 * time: 10.107649087905884
    13    -6.211077e+01     1.219386e-02
 * time: 10.749650955200195
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671062
    AtomicLocal         -18.8557648
    AtomicNonlocal      14.8522628
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336815

    total               -62.261666461869
